# Geometry
using the RealGeometry class

In [ ]:
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

sys.path.append('..')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from real_geometry import RealGeometry, glaciers
from ideal_geometry import IdealGeometry

## idealized test geometries

In [ ]:
f, ax = plt.subplots(6,3, figsize=(8,8), sharex=True, sharey=True)
for i in range(3):
    ax[-1,i].set_xlabel('x [km]')
    tg = IdealGeometry(f'test{i+1}').create()
    tg = tg.assign_coords({'x':tg.x/1e3, 'y':tg.y/1e3})
    tg.draft  .plot(ax=ax[0,i], vmin=-1000, vmax=-500)
    tg.mask   .plot(ax=ax[1,i])
    tg.alpha  .plot(ax=ax[2,i], vmin=0, vmax=0.006)
    tg.dgrl   .plot(ax=ax[3,i])
    tg.grl_adv.plot(ax=ax[4,i], vmin=-1000, vmax=-500)
    tg.box    .plot(ax=ax[5,i])
for j in range(5):
    ax[j,0].set_ylabel('y [km]')
    

In [ ]:
tg.area_k

## realistic geometries

In [ ]:
glaciers

In [ ]:
ds = xr.open_dataset('../../results/PICOP/Totten_n3_geometry.nc')

In [ ]:
da = (ds.draft.rolling(x=5).mean()+ds.draft.rolling(y=5).mean())/2
dx, dy = da.x[1]-da.x[0], da.y[1]-da.y[0]
dxdy = abs((da.y-da.y.shift(y=1))*(da.x-da.x.shift(x=1)))
ip1 = da.shift(x=-1)
im1 = da.shift(x= 1)
jp1 = da.shift(y=-1)
jm1 = da.shift(y= 1)
n1 = np.array([-2*dy*(ip1-im1), -2*dx*(jp1-jm1), 4*dxdy])
n1_norm = np.linalg.norm(n1, axis=0)

F = (ds.grl_adv.rolling(x=5).mean()+ds.grl_adv.rolling(x=5).mean())/2
grad = np.gradient(F, dx.values)
dFdx = xr.DataArray(data=grad[1], dims=da.dims, coords=da.coords)
dFdy = xr.DataArray(data=grad[0], dims=da.dims, coords=da.coords)
n2 = np.array([-dFdy, dFdx, xr.zeros_like(dFdx)])
n2_norm = np.linalg.norm(n2, axis=0)

alpha = abs(np.rad2deg(np.arccos((-dFdy*n1[0]+dFdx*n1[1])/n1_norm/n2_norm))-90)
beta  = np.rad2deg(np.arccos(4*dxdy/n1_norm))

In [ ]:
del alpha

In [ ]:
alpha.plot(vmax=10)

In [ ]:
np.deg2rad(alpha.median())

In [ ]:
np.gradient(np.array([1,2,3,np.nan,np.nan,4,5,6]))

In [ ]:
beta.plot(vmax=10)

In [ ]:
(beta-alpha).plot()

In [ ]:
ddx = da.copy()
ddy = da.copy()
ddx.data = np.gradient(da)[1]
ddy.data = np.gradient(da)[0]

fig, ax = plt.subplots(1,3, figsize=(12,5))
da.plot(ax=ax[0])
ddx.plot(ax=ax[1])
ddy.plot(ax=ax[2])


In [ ]:
plt.imshow(np.gradient(da)[0])

## plots

In [ ]:
for glacier in ['Amery', 'Totten', 'MoscowUniversity', 'Dotson', 'Thwaites', 'PineIsland']:
    RealGeometry(name=glacier).plot_PICO()

In [ ]:
for glacier in ['Amery', 'MoscowUniversity', 'Dotson', 'Thwaites', 'Totten', 'PineIsland']:
    RealGeometry(name=glacier).plot_PICOP()